In [1]:
import time
import numpy as np
from scipy.signal import butter, filtfilt
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from joblib import Parallel, delayed
import multiprocessing
from time import time
import gc
import scipy.io as sio
import glob


from MUA_helper import (make_NP_array_from_linear_data, cleanAxes, placeAxesOnGrid,filter_detect,butter_bandpass,
                        butter_bandpass_filter,basic_peak_detector,get_prepare_PSTHs,plot_save,plot_save_single)

In [2]:
def get_spiketimes(raw_data_path, data_folder,timestamps, thresh=-5):
                           
    print ("create a memmap")
    
    for trial in range(len(timestamps)):
        
        mm = np.memmap(raw_data_path, dtype=np.int16, mode='r')    
        start = int(timestamps[trial])
        end = int(start+ sweeplength)



        mm = mm[start*sampling_rate*n_ch:end*sampling_rate*n_ch]
        mm = mm.reshape((n_ch,-1), order='F')
        print( mm.shape)

        slices = np.arange(0, mm.shape[1], sweeplength*sampling_rate)
        slices = np.append(slices, mm.shape[1])

        #print ("delet the memmap ...")
        #del mm

        #print ("extract spiketimes in {} slices of 10 min each".format(str((len(slices)-1))))

        for i in range(len(slices)-1):

            save_to = data_folder+'{0:03d}_{0:03d}.npy'.format(trial,i)

            if os.path.isfile(save_to):
                print ("%d. slice already exists"%i)
                continue 

            start, stop = slices[i:i+2]

            start = int(start)
            stop = int(stop)

            print ("start the %d. slice"%i)
            startTime = time()
            spiketimes = in_the_loop(mm,start, stop,thresh)
            print ("for one slice: %.2f"%((time()-startTime)))

            np.save(save_to, np.array(spiketimes))
            print ("%d. slice saved"%i)

            gc.collect()
       
    print ("done for the whole recording")

#%%
def in_the_loop(mm, start, stop,thresh):
    gc.collect()
    
    #print ("create a memmap")
    #mm = np.memmap(raw_data_path, dtype=np.int16, mode='r')    
    #mm = mm.reshape((n_ch,-1), order='F')
    
    print ("load data into RAM ...")
    data = mm[:, start:stop].copy()
    print (data.shape)
    
    print ("delet the memmap ...")
    del mm    
    
    #print ("subtract the common average reference ... ") 
#    startTime = time()
    #data -= np.median(data, axis=0).astype(int)[np.newaxis,:]
#    print "vectorized: %.2f"%(time()-startTime)
    
    #print ("offset correction for Phase3A probes ...")
#    startTime = time()
    #data -= np.median(data, axis=1).astype(int)[:,np.newaxis]
#    print "vectorized: %.2f"%(time()-startTime)
    
    # scaling factor of the probe (irrelevant with relative threshold)
#    data = data * 0.195 
       
    print ("filter and detect spikes ..." )       
    spiketimes = Parallel(n_jobs=num_cores, verbose=10)(delayed(filter_detect)(data[ch,:], start, sampling_rate, thresh) for ch in range(n_ch))


    return spiketimes

In [3]:
## VPL I && TEMP

structure='VPL'
exp_name = 'SNA_035184_probe1_002'
working_dir =r"Z:\Neuropixel\Thalamus\SNA-035184\2019-07-18_10-52-01"

sampling_rate = 30000
thresh = -5
slicelength = 1 # in sec
num_cores = 10
n_ch = 384


exp_name_list = exp_name.split("_")

metadata_path = (glob.glob(working_dir+'/NationalInstruments/*sweepParameter'))
metadata_path_names = [os.path.basename(path) for path in metadata_path ]
stimdata_path = (glob.glob(working_dir+'/NationalInstruments/*stimData*.pkl'))
stimdata_path_names = [os.path.basename(path) for path in stimdata_path ]

if "001" in exp_name_list:
    exp_id = "001"
elif "002" in exp_name_list:
    exp_id = "002"
else:
    print('exp Id input does not match dir content')
    
if "probe1" in exp_name_list:
    probe = "probe1"
    p=0
elif "probe2" in exp_name_list:
    probe = "probe2"
    p=2
else:
    print('probe input does not match dir content')
    
for i in range(len(metadata_path_names)):
    if exp_id in metadata_path_names[i]:
        metaData = metadata_path_names[i]
        
for i in range(len(stimdata_path_names)):
    if exp_id in stimdata_path_names[i] and probe in stimdata_path_names[i] :
        stim_name = stimdata_path_names[i]

stim_data_path =os.path.join(working_dir,'NationalInstruments',stim_name)
raw_data_path = os.path.join(working_dir,'experiment1\\recording1\\continuous\\Neuropix-PXI-100.{}\\continuous.dat'.format(p))

metaData_path =os.path.join(working_dir,'NationalInstruments',metaData)
meta_tmp = pd.read_pickle(metaData_path)

sweeplength = meta_tmp.Sweeplength[0]
pre = meta_tmp['Pre Stimulus Time'][0]
trials = meta_tmp.Repititions[0]


print(num_cores)
startTime = time()

with open(stim_data_path, 'rb') as f:
    x = pickle.load(f)

try:
    timestamps = x["Stimstart"].values
except:
    timestamps = x['Timestamps'].dropna().values[::2]
    
for i in range(len(timestamps)):
    
    data_folder = working_dir + '/MUA-Analysis/' + exp_name + '/'+str(thresh) + '/timestamps_{}/'.format(i)
    if not os.path.isdir(data_folder):
        os.makedirs(data_folder)

    get_spiketimes(raw_data_path, data_folder,timestamps[i]-(pre), thresh=thresh)

endTime = time()

10
create a memmap
(384, 600000)
start the 0. slice
load data into RAM ...
(384, 600000)
delet the memmap ...
filter and detect spikes ...


[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    9.6s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   18.8s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   18.9s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   19.0s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:   19.0s
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   19.1s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1957s.) Setting batch_size=2.
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:   19.2s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1802s.) Setting batch_size=4.
[Parallel(n_jobs=10)]: Done  85 tasks      | elapsed:   19.4s
[Parallel(n_jobs=10)]: Done 119 tasks      | elapsed:   19.7s
[Parallel(n_jobs=10)]: Done 179 tasks      | elapsed:   20.1s
[Parallel(n_jobs=10)]: Done 247 tasks      | elapsed:   20.6s
[Parallel(n_jobs=10)]: Done 384 out of 384 | elapsed:   21.6s finished


for one slice: 28.05
0. slice saved
(384, 600000)
start the 0. slice
load data into RAM ...
(384, 600000)
delet the memmap ...
filter and detect spikes ...


[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    9.4s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   18.7s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   18.7s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   18.8s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:   18.9s
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   19.0s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1963s.) Setting batch_size=2.
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:   19.1s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1782s.) Setting batch_size=4.
[Parallel(n_jobs=10)]: Done  85 tasks      | elapsed:   19.2s
[Parallel(n_jobs=10)]: Done 119 tasks      | elapsed:   19.5s
[Parallel(n_jobs=10)]: Done 179 tasks      | elapsed:   20.0s
[Parallel(n_jobs=10)]: Done 247 tasks      | elapsed:   20.5s
[Parallel(n_jobs=10)]: Done 384 out of 384 | elapsed:   21.4s finished


for one slice: 27.87
0. slice saved
(384, 600000)
start the 0. slice
load data into RAM ...
(384, 600000)
delet the memmap ...
filter and detect spikes ...


...

for one slice: 28.12
0. slice saved
(384, 600000)
start the 0. slice
load data into RAM ...
(384, 600000)
delet the memmap ...
filter and detect spikes ...


[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    9.5s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   18.7s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   18.8s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   18.9s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:   18.9s
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   19.0s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1894s.) Setting batch_size=2.
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:   19.1s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1752s.) Setting batch_size=4.
[Parallel(n_jobs=10)]: Done  85 tasks      | elapsed:   19.3s
[Parallel(n_jobs=10)]: Done 119 tasks      | elapsed:   19.6s
[Parallel(n_jobs=10)]: Done 179 tasks      | elapsed:   20.0s
[Parallel(n_jobs=10)]: Done 247 tasks      | elapsed:   20.4s
[Parallel(n_jobs=10)]: Done 384 out of 384 | elapsed:   21.4s finished


for one slice: 28.19
0. slice saved
(384, 600000)
start the 0. slice
load data into RAM ...
(384, 600000)
delet the memmap ...
filter and detect spikes ...


[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    9.4s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   18.8s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   18.8s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   18.9s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:   19.0s
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   19.1s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1954s.) Setting batch_size=2.
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:   19.2s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1812s.) Setting batch_size=4.
[Parallel(n_jobs=10)]: Done  85 tasks      | elapsed:   19.3s
[Parallel(n_jobs=10)]: Done 119 tasks      | elapsed:   19.6s
[Parallel(n_jobs=10)]: Done 179 tasks      | elapsed:   20.1s
[Parallel(n_jobs=10)]: Done 247 tasks      | elapsed:   20.6s
[Parallel(n_jobs=10)]: Done 384 out of 384 | elapsed:   21.5s finished


for one slice: 28.41
0. slice saved
done for the whole recording
